# Chemical Reactions Example

This notebook demonstrates chemical reaction modeling in PHASTA, including combustion, mixing, and reaction kinetics.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from phasta import FlowSolver, FlowConfig, Mesh, ChemistryConfig

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')

## Problem Setup

We'll simulate a premixed flame with the following parameters:
- Domain size: 0.1m × 0.02m
- Fuel: Methane (CH₄)
- Oxidizer: Air (O₂ + N₂)
- Equivalence ratio: 1.0
- Inlet temperature: 300K
- Inlet velocity: 0.5 m/s

In [ ]:
# Create base configuration
config = FlowConfig()
config.domain = {
    'width': 0.1,
    'height': 0.02,
    'mesh_size': 0.0002
}

config.flow = {
    'inlet_velocity': 0.5,
    'inlet_temperature': 300.0,
    'time_step': 1e-6,
    'max_time': 0.01
}

config.boundary_conditions = {
    'inlet': 'velocity',
    'outlet': 'outflow',
    'walls': 'adiabatic'
}

## Premixed Flame Simulation

Let's simulate a premixed methane-air flame using detailed chemistry.

In [ ]:
# Create mesh
mesh = Mesh.generate_rectangle_2d(
    width=config.domain['width'],
    height=config.domain['height'],
    nx=500,
    ny=100
)

# Plot mesh
plt.figure(figsize=(10, 2))
mesh.plot()
plt.title('Premixed Flame Mesh')
plt.show()

In [ ]:
# Configure chemistry
chemistry_config = ChemistryConfig()
chemistry_config.mechanism = 'gri30'
chemistry_config.species = [
    'CH4', 'O2', 'N2', 'CO2', 'H2O', 'CO', 'H2',
    'OH', 'H', 'O', 'HO2', 'H2O2', 'CH3', 'CH2O'
]
chemistry_config.reactions = 'all'
chemistry_config.transport = 'multicomponent'
chemistry_config.radiation = 'optically_thin'

# Set initial conditions
chemistry_config.initial_conditions = {
    'temperature': 300.0,
    'pressure': 101325.0,
    'equivalence_ratio': 1.0,
    'fuel': 'CH4',
    'oxidizer': 'O2'
}

# Create and run solver
solver = FlowSolver(config, mesh, chemistry_config=chemistry_config)
results = solver.solve()

print("\nPremixed Flame Results:")
print(f"Simulation completed: {results.completed}")
print(f"Final time: {results.final_time}")
print(f"Number of time steps: {results.time_steps}")

## Visualize Results

Plot temperature, species mass fractions, and reaction rates.

In [ ]:
plt.figure(figsize=(15, 5))

# Plot temperature field
plt.subplot(131)
results.plot_temperature()
plt.title('Temperature Field')
plt.colorbar(label='Temperature (K)')

# Plot fuel mass fraction
plt.subplot(132)
results.plot_species_mass_fraction('CH4')
plt.title('CH₄ Mass Fraction')
plt.colorbar(label='Mass Fraction')

# Plot OH mass fraction
plt.subplot(133)
results.plot_species_mass_fraction('OH')
plt.title('OH Mass Fraction')
plt.colorbar(label='Mass Fraction')

plt.tight_layout()
plt.show()

In [ ]:
# Plot reaction rates and heat release
plt.figure(figsize=(15, 5))

# Plot heat release rate
plt.subplot(131)
results.plot_heat_release_rate()
plt.title('Heat Release Rate')
plt.colorbar(label='Heat Release (W/m³)')

# Plot CH₄ consumption rate
plt.subplot(132)
results.plot_reaction_rate('CH4')
plt.title('CH₄ Consumption Rate')
plt.colorbar(label='Rate (mol/m³/s)')

# Plot OH production rate
plt.subplot(133)
results.plot_reaction_rate('OH')
plt.title('OH Production Rate')
plt.colorbar(label='Rate (mol/m³/s)')

plt.tight_layout()
plt.show()

## Non-Premixed Flame Simulation

Now let's simulate a non-premixed flame with separate fuel and oxidizer streams.

In [ ]:
# Create mesh for non-premixed flame
mesh = Mesh.generate_rectangle_2d(
    width=0.1,
    height=0.02,
    nx=500,
    ny=100
)

# Plot mesh
plt.figure(figsize=(10, 2))
mesh.plot()
plt.title('Non-Premixed Flame Mesh')
plt.show()

In [ ]:
# Configure non-premixed chemistry
chemistry_config = ChemistryConfig()
chemistry_config.mechanism = 'gri30'
chemistry_config.species = [
    'CH4', 'O2', 'N2', 'CO2', 'H2O', 'CO', 'H2',
    'OH', 'H', 'O', 'HO2', 'H2O2', 'CH3', 'CH2O'
]
chemistry_config.reactions = 'all'
chemistry_config.transport = 'multicomponent'
chemistry_config.radiation = 'optically_thin'
chemistry_config.mixture_fraction = True

# Set initial conditions
chemistry_config.initial_conditions = {
    'temperature': 300.0,
    'pressure': 101325.0,
    'fuel_stream': {
        'species': 'CH4',
        'temperature': 300.0,
        'velocity': 0.5
    },
    'oxidizer_stream': {
        'species': 'O2',
        'temperature': 300.0,
        'velocity': 0.5
    }
}

# Create and run solver
solver = FlowSolver(config, mesh, chemistry_config=chemistry_config)
results = solver.solve()

print("\nNon-Premixed Flame Results:")
print(f"Simulation completed: {results.completed}")
print(f"Final time: {results.final_time}")
print(f"Number of time steps: {results.time_steps}")

## Visualize Non-Premixed Results

Plot mixture fraction, temperature, and species mass fractions.

In [ ]:
plt.figure(figsize=(15, 5))

# Plot mixture fraction
plt.subplot(131)
results.plot_mixture_fraction()
plt.title('Mixture Fraction')
plt.colorbar(label='Mixture Fraction')

# Plot temperature field
plt.subplot(132)
results.plot_temperature()
plt.title('Temperature Field')
plt.colorbar(label='Temperature (K)')

# Plot OH mass fraction
plt.subplot(133)
results.plot_species_mass_fraction('OH')
plt.title('OH Mass Fraction')
plt.colorbar(label='Mass Fraction')

plt.tight_layout()
plt.show()

In [ ]:
# Plot reaction rates and heat release
plt.figure(figsize=(15, 5))

# Plot heat release rate
plt.subplot(131)
results.plot_heat_release_rate()
plt.title('Heat Release Rate')
plt.colorbar(label='Heat Release (W/m³)')

# Plot CH₄ consumption rate
plt.subplot(132)
results.plot_reaction_rate('CH4')
plt.title('CH₄ Consumption Rate')
plt.colorbar(label='Rate (mol/m³/s)')

# Plot OH production rate
plt.subplot(133)
results.plot_reaction_rate('OH')
plt.title('OH Production Rate')
plt.colorbar(label='Rate (mol/m³/s)')

plt.tight_layout()
plt.show()

## Exercises

1. Try different equivalence ratios in premixed flames
2. Experiment with different fuel-oxidizer combinations
3. Study the effect of pressure on flame speed
4. Compare different chemical mechanisms

## Next Steps

- Try the particle-laden flow example
- Explore phase change phenomena
- Learn about turbulent combustion